<a href="https://colab.research.google.com/github/Joaogmr472/GLM/blob/main/C%C3%B3digo_3_plota_espacial_glm_ELISA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **GOES-16: REDE GLM (PARTE III)**
**Plota mapas de flashes do GLM (GOES-16) para o Brasil**

---
**OBJETIVO:** Este Google Colab plota mapas anual e mensal para o Brasil de flashes do GLM.

**DADOS DE ENTRADA**: Dados de flashes do GOES-16. 
Dados de flashes horários gerados a partir dos dados do CPTEC. Estão na pasta **output** do drive.. **Nome:** flash_diario_goes16_2020-01-03.nc. 
---
**DADOS DE SAÍDA:**
1.   Mapa **anual** de flash. 
2.   Mapa **mensal** de flash. 
3.   Mapa **sazonal** de flash. 
---
**PROCEDIMENTO REALIZADO:** Os seguintes procedimentos são realizados nesse código:
1.   Instalação de bibliotecas
2.   Importação das bibliotecas
3.   Monta o drive da pessoa
4.   Download dos dados do Diego, através do Dropbox
5.   Descompacta os dados baixados
6.   Acumula os flashes por dia
7.   Salva os dados diário num arquivo netcdf
 
---
**OBSERVAÇÕES IMPORTANTES**: 
1. Mudar o caminho para seu drive. Veja em: **###MUDANÇAS AQUI####**
---
***Realizado por: [João G M Ribeiro](https://github.com/Joaogmr472/) e [Enrique V Mattos](https://github.com/evmpython) - 07/mar/2022*** 

# **###MUDANÇAS AQUI###**

- Insira o ano desejado

In [ ]:
ano = '2021'

- Insira o caminho do seu drive

In [ ]:
# Pasta com os dados Horários
path = f'/content/drive/MyDrive/JOAO_GABRIEL_MESTRADO/MESTRADO/DADOS/GLM/DADOS_horarios/{ano}/'
# Pasta para a Saída das Figuras
path_fig = f'/content/drive/MyDrive/JOAO_GABRIEL_MESTRADO/MESTRADO/DADOS/GLM/figuras/'

# **Instalando as bibliotecas**

In [ ]:
# Instalando o cartopy (usado para gerar Mapas)
!apt-get install libproj-dev proj-data proj-bin
!apt-get install libgeos-dev
!pip install --no-binary shapely shapely --force
!pip install cartopy

# Instalando o proplot (usado para gerar Mapas também)
!pip install proplot==0.6.4

# Intalando pacote que descompacta arquivos .rar
!pip install patool pyunpack

# Instalando o Salem (usado para mascarar regiões)
!pip install rasterio
!pip install pyproj
!pip install geopandas
!pip install salem
print('\n')

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libproj-dev is already the newest version (4.9.3-2).
libproj-dev set to manually installed.
proj-data is already the newest version (4.9.3-2).
proj-data set to manually installed.
The following package was automatically installed and is no longer required:
  libnvidia-common-470
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  proj-bin
0 upgraded, 1 newly installed, 0 to remove and 39 not upgraded.
Need to get 32.3 kB of archives.
After this operation, 110 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 proj-bin amd64 4.9.3-2 [32.3 kB]
Fetched 32.3 kB in 1s (46.5 kB/s)
Selecting previously unselected package proj-bin.
(Reading database ... 155320 files and directories currently installed.)
Preparing to unpack .../proj-bin_4.9.3-2_amd64.deb ...
Unpacking proj-bin (4.9.3-2) ...
Setting up proj-bin (4.9.3-2

# **Importa as bibliotecas**

In [ ]:
import xarray as xr
import proplot as plot
import time as t
import pandas as pd
from datetime import datetime
from tqdm.notebook import tqdm_notebook
import glob
import numpy as np
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
from pyunpack import Archive
import os
from time import process_time
import calendar
import salem
from tqdm.notebook import tqdm_notebook

import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.7/dist-packages/proplot/config.py:1454: ProPlotWarning: Rebuilding font cache.


# **Monta Drive**

In [ ]:
# acessando o drive
from google.colab import drive
drive.mount('/content/drive')

# **Download de informações auxiliares**

In [ ]:
# Criando os diretórios de saida
os.makedirs(f'{path}output/figuras', exist_ok=True)

# Baixando os arquivos de shapefile de alguns estados
!wget -c https://geoftp.ibge.gov.br/organizacao_do_territorio/malhas_territoriais/malhas_municipais/municipio_2020/Brasil/BR/BR_UF_2020.zip

!unzip -o /content/BR_UF_2020.zip


# **Importa funções**

In [ ]:
#----------------------------------------------------------------------------------- 
# Função que plota por Estado 
#----------------------------------------------------------------------------------- 
def evm_plot_states():
    shapefile = list(shpreader.Reader('/content/BR_UF_2020.shp').geometries()) ; ax.add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='black',facecolor='none', linewidth=1.7)   


# **Latitudes e Longitudes limitantes da América do Sul**

In [ ]:
# lats/lons do quadrado limitando a América do Sul
latmin, latmax, lonmin, lonmax = -60, 15, -90, -30

# gera a mascara para ro Brasil
shp_br = salem.read_shapefile(f'{path_fig}brazil-coastline/brazil_coastline.shp')
#shp_NEB = shp.loc[shp['nome'] == 'Nordeste']

**Acumula os dados de flashes por ano**

# **Plota a Figura Mensal**

In [ ]:
# cria a moldura da figura   
fig, ax = plot.subplots(axwidth=3, axheight=3, ncols=3, nrows=4, tight=True, proj='pcarree')

# formatação dos eixos
ax.format(coast=True, borders=True, innerborders=True, labels=True, latlines=5, lonlines=3,
          suptitle=f'Monthly Flashes GOES-16 (GLM): {ano}', 
          latlim=(latmin, latmax), lonlim=(lonmin, lonmax), fontsize=11, 
          abc='(A)', small='25px', large='25px')

# plota estados
evm_plot_states()

# loop dos meses

soma_mens = np.zeros((12,1664,1664))
for i in range(0, 12):
  
  mes = str(i+1).zfill(2)
  
  print('PROCESSANDO MÊS --->>>', mes)
    
  # loop dos arquivos daquele mês
  soma_mes = np.zeros((1664,1664))
  files = glob.glob(f'{path}flash_horario_goes16_{ano}-{mes}*nc')
  files = sorted(files)
  for file in tqdm_notebook(files):
        
    # leitura do arquivo netcdf
    dados = xr.open_dataset(file)   

    # somando os flashes                   
    soma_mes = soma_mes + dados['flash'][0,:,:]
        
  # plota figura
  if i == 0: 
    #.salem.roi(shape=shp_br)
    map1 = ax[i].contourf(soma_mes['lon'], soma_mes['lat'], (soma_mes/64.), cmap='jet', levels=plot.arange(0., 10, 1))
  else:
    ax[i].contourf(soma_mes['lon'], soma_mes['lat'], (soma_mes/64.), cmap='jet', levels=plot.arange(0., 10, 1)) 
        
  # plota titulo de cada figura
  ax[i].format(title=calendar.month_name[int(mes)])

  soma_mens[i,:,:] = soma_mes[:,:] 

# plota barra de cores da figura
fig.colorbar(map1, loc='b', label='Flash Rate Density $(fl.km^{-2}.month^{-1})$', ticks=2, ticklabelsize=20, labelsize=20, space=0.4, length=0.60, width=0.3)

# salva figura
fig.save(f'{path_fig}Fig2_glm_{ano}_mensal.jpg', dpi=300) 

#**Plota a Figura Anual**

## Cria o Acumulado Anual

In [ ]:
soma_mens

In [ ]:
soma_anual = np.zeros((1664,1664))
for file in tqdm_notebook(glob.glob(f'{path}flash_horario_goes16_{ano}*nc')):
  dset = xr.open_dataset(file)
  soma_anual = soma_anual + dset['flash'][0,:,:]

In [ ]:
files = glob.glob(f'{path}flash_horario_goes16_{ano}*nc')
dset = xr.open_dataset(files[0])
dset

## Plota a Figura

In [ ]:
# Plotando a figura
fig, ax = plot.subplots(figsize=(12,12), tight=True, proj='pcarree')

# define formato da figura
ax.format(coast=False, borders=False, innerborders=False, labels=True, latlines=5, lonlines=5,
          latlim=(latmin, latmax), lonlim=(lonmin, lonmax), fontsize=11, 
          suptitle=f'Anual Flashes GOES-16 (GLM): {ano}', small='20px', large='20px', abc=True)

# plota contornos dos Estados
evm_plot_states()

# Plota Figura Figura
map = ax.contourf(soma_anual['lon'], soma_anual['lat'], (soma_anual/64.).salem.roi(shape=shp_NEB), cmap='jet', vmax=30)

# plota barra de cores da figura
fig.colorbar(map, loc='b', label='Flash Rate Density $(fl.km^{-2}.year^{-1})$', ticks=5, ticklabelsize=13, labelsize=15, space=0.4, length=0.93)

# salva figura
fig.save(f'{path_fig}/Fig1_glm_{ano}_anual.jpg', dpi=300)